Preprocessing
This notebook manipulates the review texts. It removes the stopwords, tokenizes each review, then stemms and lemmatizes the tokens. Next with nltk package sentiment of each review is determined. Last a tf-idf and Bag-of-Words algorithm is implemented. The resulting files are saved to data/intermediate

In [ ]:
import os
import json
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer
from joblib import Parallel, delayed  # For parallel processing

In [ ]:
# Ensure necessary NLTK resources are downloaded
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("vader_lexicon")

# Initialize stemmer, lemmatizer, and sentiment analyzer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()

In [ ]:
# --- File Paths Setup ---
# Base directory (go 2 levels up from /src1/)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Define input/output directories using relative paths
input_dir = os.path.join(base_dir, "data", "intermediate")
output_dir = os.path.join(base_dir, "data", "intermediate")

input_filename = "cleaned_reviews_2021-01.json"  # Adjust as needed
input_file = os.path.join(input_dir, input_filename)

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Function to read JSON file line by line
def load_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        reviews = [json.loads(line) for line in file]
    return pd.DataFrame(reviews)


# Function for cleaning, tokenizing, and stemming/lemmatizing text


def preprocess_text_dict(text):
    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming and lemmatization
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    return {
        "tokens": filtered_tokens,
        "stemmed": stemmed_tokens,
        "lemmatized": lemmatized_tokens,
    }

In [ ]:
# Sentiment Analysis Function
def analyze_sentiment(text):
    scores = sia.polarity_scores(text)
    if scores["compound"] >= 0.05:
        return "positive"
    elif scores["compound"] <= -0.05:
        return "negative"
    else:
        return "neutral"

TF-IDF Function

In [ ]:
# TF-IDF Vectorization function
def process_tfidf(df):
    # Apply preprocessing and lemmatization
    df["processed_text"] = df["text"].apply(preprocess_text_dict)
    df["lemmatized_text"] = df["processed_text"].apply(
        lambda x: " ".join(x["lemmatized"])
    )

    # Apply optimized TF-IDF Vectorization
    vectorizer = TfidfVectorizer(
        max_features=5000,  # Limit vocabulary size for efficiency
        sublinear_tf=True,  # Scale term frequency logarithmically
        max_df=0.95,  # Ignore very common words
        min_df=5,  # Ignore very rare words
        ngram_range=(1, 2),  # Consider unigrams and bigrams
    )

    tfidf_matrix = vectorizer.fit_transform(df["lemmatized_text"])

    # Convert TF-IDF matrix to DataFrame efficiently
    tfidf_df = pd.DataFrame(
        tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out()
    )

    return df, tfidf_df

In [ ]:
# Save processed data to files
def save_processed_data(df, tfidf_df, bow_df):
    # Save processed reviews
    df[["review_id", "processed_text"]].to_json(
        os.path.join(output_dir, "processed_reviews_tokens.json"),
        orient="records",
        lines=True,
    )

    # Save lemmatized text
    df[["review_id", "lemmatized_text"]].to_json(
        os.path.join(output_dir, "lemmatized_text.json"), orient="records", lines=True
    )

    # Save TF-IDF scores
    tfidf_df.to_csv(os.path.join(output_dir, "tfidf_scores.csv"), index=False)

    # Save BoW vectors
    bow_df.to_csv(os.path.join(output_dir, "bow_vectors.csv"), index=False)

    print("Processed data saved to files.")

In [ ]:
# Load the dataset
df = load_dataset(input_file)

# Apply Sentiment Analysis
df["sentiment"] = df["text"].apply(analyze_sentiment)
df.to_json(
    os.path.join(output_dir, "reviews_with_sentiment.json"),
    orient="records",
    lines=True,
)

In [ ]:
# Process TF-IDF (Takes a while to run)
processed_df, tfidf_df = process_tfidf(df)
print(processed_df[["review_id", "text", "sentiment"]].head())
print(tfidf_df.head())

Bag of Words Function

In [ ]:
with open(input_file, "r") as file:
    data = [json.loads(line) for line in file]

# Extract the relevant fields
reviews = [{"text": entry["text"], "stars": entry["stars"]} for entry in data]

# Preprocess the text
stemmer = SnowballStemmer("english")  # Faster stemmer


def preprocess_text_string(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenize and stem
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(stemmed_tokens)


# Parallelize text preprocessing
print("Preprocessing text...")
texts = Parallel(n_jobs=-1)(
    delayed(preprocess_text_string)(review["text"]) for review in reviews
)

# Label the stars


def label_stars(stars):
    if 1 <= stars <= 2:
        return 0  # Negative
    elif 4 <= stars <= 5:
        return 1  # Positive
    else:
        return 2  # Neutral


print("Labeling stars...")
sentiment_labels = [label_stars(review["stars"]) for review in reviews]

# Create the Bag-of-Words representation
print("Creating Bag-of-Words representation...")
vectorizer = CountVectorizer(
    max_features=10000, ngram_range=(1, 1), binary=True
)  # Limit vocabulary size and use binary features
bow_matrix = vectorizer.fit_transform(texts)  # This is a sparse matrix

# Convert the BoW matrix to a DataFrame (sparse format)
bow_df = pd.DataFrame.sparse.from_spmatrix(
    bow_matrix, columns=vectorizer.get_feature_names_out()
)

# Add the 'text' and 'sentiment label' columns to the DataFrame
bow_df["text"] = texts  # Add the preprocessed text column
bow_df["sentiment label"] = sentiment_labels  # Add the sentiment label column

# Reorder columns to have 'text' and 'sentiment label' as the first two columns
cols = ["text", "sentiment label"] + [
    col for col in bow_df.columns if col not in ["text", "sentiment label"]
]
bow_df = bow_df[cols]

# Save the result to a CSV file
print("Saving to CSV...")
output_file = os.path.join(output_dir, "bow_vector.csv")
bow_df.to_csv(output_file, index=False)

print("Bag-of-Words representation saved to 'reviews_bow.csv'")

In [ ]:
save_processed_data(processed_df, tfidf_df, bow_df)